# Загрузка библиотек

In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import joblib
import warnings
warnings.filterwarnings('ignore')  # Игнорируем предупреждения для чистоты вывода

# === 1. Загрузка данных ===

In [100]:
data = pd.read_csv("./data/emails_clean.csv")

print("🔎 Пример данных:")
print(data.head(), "\n")

🔎 Пример данных:
                                          clean_text  spam
0  naturally irresistible your corporate identity...     1
1  the stock trading gunslinger fanny is merrill ...     1
2  unbelievable new homes made easy im wanting to...     1
3  color printing special request additional info...     1
4  do not have money get software cds from here s...     1 



In [101]:
# Проверка баланса классов
print("Баланс классов (spam/ham):")
print(data['spam'].value_counts(normalize=True), "\n")

Баланс классов (spam/ham):
spam
0    0.761173
1    0.238827
Name: proportion, dtype: float64 



# Векторизация

In [102]:
vectorizer = TfidfVectorizer(
    stop_words='english',   # удаляем стоп-слова
    max_features=5000,     # ограничиваем число признаков
    ngram_range=(1, 2),  # используем униграммы и биграммы
    max_df=0.85,
    min_df=3,
    lowercase=True,
    sublinear_tf=True
)

X = vectorizer.fit_transform(data['clean_text'])
y = data['spam']

print("✅ Матрица TF-IDF создана:", X.shape)

✅ Матрица TF-IDF создана: (5728, 5000)


# Разбиение данных

In [103]:
# === Train/Test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)

Размер обучающей выборки: (4296, 5000)
Размер тестовой выборки: (1432, 5000)


# Обработка дисбаланса

In [104]:
from imblearn.over_sampling import SMOTE

# Применение SMOTE после разделения данных
smote = SMOTE(random_state=42, sampling_strategy='auto')  # 'auto' создаёт равное число примеров для обоих классов
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("Размер сбалансированной обучающей выборки:", X_train_resampled.shape)
print("Баланс классов после SMOTE:")
print(pd.Series(y_train_resampled).value_counts(normalize=True), "\n")

Размер сбалансированной обучающей выборки: (6540, 5000)
Баланс классов после SMOTE:
spam
0    0.5
1    0.5
Name: proportion, dtype: float64 



# === Обучение ===

In [105]:
knn_params = {
    'n_neighbors': [3, 5, 7, 10],
    'metric': ['cosine']
}

knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=3, scoring='f1', n_jobs=-1)
knn_grid.fit(X_train_resampled, y_train_resampled)

y_pred_knn = knn_grid.predict(X_test)

print("=== k-NN Classifier ===")
print("Лучшие параметры:", knn_grid.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))

=== k-NN Classifier ===
Лучшие параметры: {'metric': 'cosine', 'n_neighbors': 3}
Accuracy: 0.979050279329609

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      1090
           1       0.94      0.97      0.96       342

    accuracy                           0.98      1432
   macro avg       0.97      0.98      0.97      1432
weighted avg       0.98      0.98      0.98      1432

Confusion Matrix:
 [[1069   21]
 [   9  333]]


# Кросс-валидация для k-NN

In [106]:
knn_scores = cross_val_score(knn_grid.best_estimator_, X, y, cv=5, scoring='accuracy')
print("Кросс-валидация (k-NN):", knn_scores.mean(), "±", knn_scores.std(), "\n")

Кросс-валидация (k-NN): 0.9884789318457212 ± 0.0032340881417894226 



# === Обучение ===

In [107]:
nb = MultinomialNB(alpha=0.1, class_prior=None)
nb.fit(X_train_resampled, y_train_resampled)

y_pred_nb = nb.predict(X_test)

print("=== Naive Bayes Classifier ===")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_nb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))

=== Naive Bayes Classifier ===
Accuracy: 0.986731843575419

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      1090
           1       0.97      0.98      0.97       342

    accuracy                           0.99      1432
   macro avg       0.98      0.98      0.98      1432
weighted avg       0.99      0.99      0.99      1432

Confusion Matrix:
 [[1079   11]
 [   8  334]]


# Кросс-валидация для Naive Bayes

In [108]:
nb_scores = cross_val_score(nb, X, y, cv=5, scoring='accuracy')
print("Кросс-валидация (Naive Bayes):", nb_scores.mean(), "±", nb_scores.std(), "\n")

Кросс-валидация (Naive Bayes): 0.987080484998133 ± 0.0016968029214997345 



# Сравнение моделей

In [109]:
# Оценка k-NN
y_pred_knn = knn_grid.predict(X_test)
print("=== k-NN Classifier ===")
print("Лучшие параметры:", knn_grid.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("F1-score:", f1_score(y_test, y_pred_knn))  # Добавляем F1
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))

# Кросс-валидация для k-NN
knn_scores = cross_val_score(knn_grid.best_estimator_, X, y, cv=3, scoring='f1')
print("Кросс-валидация F1 (k-NN):", knn_scores.mean(), "±", knn_scores.std(), "\n")

# Оценка Naive Bayes
y_pred_nb = nb.predict(X_test)
print("=== Naive Bayes Classifier ===")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("F1-score:", f1_score(y_test, y_pred_nb))  # Добавляем F1
print("\nClassification Report:\n", classification_report(y_test, y_pred_nb, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))

# Кросс-валидация для Naive Bayes
nb_scores = cross_val_score(nb, X, y, cv=3, scoring='f1')
print("Кросс-валидация F1 (Naive Bayes):", nb_scores.mean(), "±", nb_scores.std(), "\n")

=== k-NN Classifier ===
Лучшие параметры: {'metric': 'cosine', 'n_neighbors': 3}
Accuracy: 0.979050279329609
F1-score: 0.9568965517241379

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      1090
           1       0.94      0.97      0.96       342

    accuracy                           0.98      1432
   macro avg       0.97      0.98      0.97      1432
weighted avg       0.98      0.98      0.98      1432

Confusion Matrix:
 [[1069   21]
 [   9  333]]
Кросс-валидация F1 (k-NN): 0.9757163675190142 ± 0.0009283212549808528 

=== Naive Bayes Classifier ===
Accuracy: 0.986731843575419
F1-score: 0.9723435225618632

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      1090
           1       0.97      0.98      0.97       342

    accuracy                           0.99      1432
   macro avg       0.98      0.98      0.98      1432
weighte

# === Сохранение модели ===

In [110]:
# Сохранение моделей и векторизатора
joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(knn_grid.best_estimator_, "knn_model.pkl")  # Сохраняем лучшую модель k-NN
joblib.dump(nb, "nb_model.pkl")
print("Модели и векторизатор сохранены!")

Модели и векторизатор сохранены!


# Mini app для проверки моделей

In [111]:
vectorizer = joblib.load("vectorizer.pkl")
knn = joblib.load("knn_model.pkl")
nb = joblib.load("nb_model.pkl")

def check_models(texts):
    X_vec = vectorizer.transform(texts)
    
    knn_pred = knn.predict(X_vec)
    nb_pred = nb.predict(X_vec)
    
    for i, text in enumerate(texts):
        print(f"\nТекст: '{text}'")
        print(f"  KNN: {knn_pred[i]}")
        print(f"  Naive Bayes: {nb_pred[i]}")
        print("-" * 30)

if __name__ == "__main__":
    print("Введите предложение для проверки (или 'exit' для выхода):")
    
    while True:
        user_input = input("> ").strip()
        if user_input.lower() == "exit":
            print("Выход из программы.")
            break
        elif user_input:
            check_models([user_input])
        else:
            print("Пустой ввод, попробуйте снова.")

Введите предложение для проверки (или 'exit' для выхода):
Выход из программы.
